In [67]:
import numpy as np 
import pandas as pd
import mysql.connector as mysql
from datetime import date
from datetime import timedelta
import datetime
import os

# LC names of last and current month
last_month_LC = "dummy.Lifecycle_LevisIn_31Mar24"
current_month_LC = "dummy.Lifecycle_LevisIn_4Apr24"

In [68]:
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")
mycursor_leviscalendar =SQLdata.cursor()

mycursor_leviscalendar.execute("""SELECT YEAR,MONTH,monthcount month_num,MIN(DATE)start_date,MAX(DATE)end_date FROM dummy.levis_calendar \
GROUP BY 1,2 ORDER BY DATE ASC;""")

calendar_df = pd.DataFrame(mycursor_leviscalendar)
calendar_df.columns = mycursor_leviscalendar.column_names

calendar_df['start_date'] = pd.to_datetime(calendar_df['start_date'])
calendar_df['end_date'] = pd.to_datetime(calendar_df['end_date'])

# Get the current month start and end dates
current_date = pd.Timestamp.now()
current_month = current_date.month
current_year = current_date.year

In [69]:
## Selected date rows
previous_dates = calendar_df[calendar_df['end_date'] < pd.Timestamp.now().strftime('%Y-%m-%d')]
sorted_dates = previous_dates.sort_values(by='end_date', ascending=False).iloc[0]
last_month_dates = pd.DataFrame([sorted_dates], columns=calendar_df.columns)
last_month_dates_LY = calendar_df[(calendar_df['YEAR'] == last_month_dates['YEAR'].iloc[0] - 1) & 
                                   (calendar_df['MONTH'] == last_month_dates['MONTH'].iloc[0])]
last_month_2Years_before_dates = calendar_df[(calendar_df['YEAR'] == last_month_dates['YEAR'].iloc[0] - 2) & 
                                   (calendar_df['MONTH'] == last_month_dates['MONTH'].iloc[0])]

sorted_dates_2M_before = previous_dates.sort_values(by='end_date', ascending=False).iloc[1]
last_2month_before_dates = pd.DataFrame([sorted_dates_2M_before], columns=calendar_df.columns)

In [70]:
## Start and End dates of months

# a & b
last_month2Yback_start_date = pd.to_datetime(last_month_2Years_before_dates['start_date'].iloc[0]).strftime('%Y-%m-%d')
last_month2Yback_end_date = pd.to_datetime(last_month_2Years_before_dates['end_date'].iloc[0]).strftime('%Y-%m-%d')
# c & d
last_month_start_date_LY = pd.to_datetime(last_month_dates_LY['start_date'].iloc[0]).strftime('%Y-%m-%d')
last_month_end_date_LY = pd.to_datetime(last_month_dates_LY['end_date'].iloc[0]).strftime('%Y-%m-%d')
# e & f
last_2month_start_date = pd.to_datetime(last_2month_before_dates['start_date'].iloc[0]).strftime('%Y-%m-%d')
last_2month_end_date = pd.to_datetime(last_2month_before_dates['end_date'].iloc[0]).strftime('%Y-%m-%d')
# g & h
last_month_start_date = pd.to_datetime(last_month_dates['start_date'].iloc[0]).strftime('%Y-%m-%d')
last_month_end_date = pd.to_datetime(last_month_dates['end_date'].iloc[0]).strftime('%Y-%m-%d')


## Tagging MMM-YY to variable
month = last_month_2Years_before_dates['MONTH'].iloc[0][:3]
year = str(last_month_2Years_before_dates['YEAR'].iloc[0])[-2:]
duration_2Y_back = month + '-' + year

month_LY = last_month_dates_LY['MONTH'].iloc[0][:3]
year_Y = str(last_month_dates_LY['YEAR'].iloc[0])[-2:]
duration_PY = month_LY + '-' + year_Y

month_LY = last_2month_before_dates['MONTH'].iloc[0][:3]
year_Y = str(last_2month_before_dates['YEAR'].iloc[0])[-2:]
duration_lasttolast_month = month_LY + '-' + year_Y

month = last_month_dates['MONTH'].iloc[0][:3]
year = str(last_month_dates['YEAR'].iloc[0])[-2:]
duration = month + '-' + year

#last_month2Yback_start_date  a
 #last_month2Yback_end_date  b
 #last_month_start_date_LY  c
  #last_month_end_date_LY  d
 #last_2month_start_date  e
  #last_2month_end_date  f
#last_month_start_date  g
 #last_month_end_date  h
    
#duration_2Y_back
#duration_PY
#duration_lasttolast_month
#duration

In [71]:
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")

## Importing 
## SKU report loyalty
mycursor_SkuDump =SQLdata.cursor()
mycursor_SkuDump.execute("""SELECT txnmappedmobile,modifiedtxndate,itemmrp,itemnetamount,dayssincelastvisit,modifiedbillno,
modifiedstorecode,frequencycount,itemqty FROM levis.sku_report_loyalty WHERE (modifiedtxndate \
BETWEEN '"""+last_month_start_date+"""' AND '"""+last_month_end_date+"""') AND \
itemqty>0;""")
skudump = pd.DataFrame(mycursor_SkuDump)
skudump.columns = mycursor_SkuDump.column_names

skudump = skudump.astype({'itemmrp' : 'float64','itemnetamount' : 'float64','dayssincelastvisit' : 'float64','frequencycount' : 'int32','itemqty' : 'int32'})

## Member report
mycursor_memberreport =SQLdata.cursor()

mycursor_memberreport.execute("""SELECT mobile,gender,enrolledstorecode,date(dateofbirth)dateofbirth,email FROM \
levis.member_report WHERE modifiedenrolledon BETWEEN '"""+last_month_start_date+"""' AND '"""+last_month_end_date+"""';""")

memberdump = pd.DataFrame(mycursor_memberreport)
memberdump.columns = mycursor_memberreport.column_names

## Sku report NonLoyalty
mycursor_NLSkuDump =SQLdata.cursor()

mycursor_NLSkuDump =SQLdata.cursor()
mycursor_NLSkuDump.execute("""SELECT modifiedtxndate,modifiedbillno,modifiedstorecode,itemnetamount 
FROM levis.sku_report_nonloyalty WHERE modifiedtxndate BETWEEN '"""+last_month_start_date+"""' AND '"""+last_month_end_date+"""' \
AND itemqty>0;""")
NLskudump = pd.DataFrame(mycursor_NLSkuDump)
NLskudump.columns = mycursor_NLSkuDump.column_names

## Store master
mycursor_storemaster =SQLdata.cursor()

mycursor_storemaster.execute("""SELECT storecode,region FROM levis.store_master group by 1;""")

storedump = pd.DataFrame(mycursor_storemaster)
storedump.columns = mycursor_storemaster.column_names

## Winback
skudump['tag'] = None
skudump.loc[(skudump['dayssincelastvisit'] >= 731) & (skudump['dayssincelastvisit'] <= 820), 'tag'] = '731-820'
skudump.loc[(skudump['dayssincelastvisit'] >= 821) & (skudump['dayssincelastvisit'] <= 910), 'tag'] = '821-910'
skudump.loc[(skudump['dayssincelastvisit'] >= 911) & (skudump['dayssincelastvisit'] <= 1000), 'tag'] = '911-1000'
skudump.loc[(skudump['dayssincelastvisit'] >= 1001) & (skudump['dayssincelastvisit'] <= 1200), 'tag'] = '1001-1200'
skudump.loc[(skudump['dayssincelastvisit'] >= 1201) & (skudump['dayssincelastvisit'] <= 1500), 'tag'] = '1201-1500'
skudump.loc[(skudump['dayssincelastvisit'] >= 1501) & (skudump['dayssincelastvisit'] <= 2000), 'tag'] = '1501-2000'
skudump.loc[skudump['dayssincelastvisit'] > 2000, 'tag'] = '2000+'

winback = skudump.groupby('tag')['txnmappedmobile'].nunique().reset_index()
winback.columns = ['tag', 'Customers']

winback_transpose = winback.set_index('tag').transpose().sort_index(ascending=False)
winback_transpose['Duration'] = duration
columns = ['Duration'] + [col for col in winback_transpose if col != 'Duration']
winback_output = winback_transpose[columns]

# Enrollments
enrollments = memberdump.shape[0]

female_enrolments = memberdump[(memberdump['gender'].str.contains('FEMALE|female', case=False, na=False)) & 
                                (memberdump['gender'].notna())]['mobile'].nunique()
                               
male_enrolments = memberdump[(memberdump['gender'].str.contains('Male|male', case=False, na=False)) & 
                              (memberdump['gender'].notna())]['mobile'].nunique()

unmapped_enrolments = memberdump[(memberdump['gender'].isna()) | (memberdump['gender'] == '')]['mobile'].nunique()

dob = memberdump[memberdump['dateofbirth'].notna()]['mobile'].nunique()

email = memberdump[memberdump['email'].notna()]['mobile'].nunique()

enrollments_op = pd.DataFrame({
    'enrollments': [enrollments],
    'female_enrolments': [female_enrolments],
    'male_enrolments': [male_enrolments],
    'unmapped_enrolments': [unmapped_enrolments],
    'dob': [dob],
    'email': [email]
})

enrollments_op['Duration'] = duration
columns = ['Duration'] + [col for col in enrollments_op if col != 'Duration']
enrollments_output = enrollments_op[columns]


# MOM metrics calculation
loyalty_customers = skudump['txnmappedmobile'].nunique()
repeaters = skudump.loc[skudump['frequencycount'] > 1, 'txnmappedmobile'].nunique()
winback_customers = skudump.loc[skudump['dayssincelastvisit'] > 730, 'txnmappedmobile'].nunique()

nonloyalty_txn = NLskudump[['modifiedtxndate', 'modifiedbillno', 'modifiedstorecode']].drop_duplicates().shape[0]
nonloyalty_sales = NLskudump['itemnetamount'].sum()

MOM_op = pd.DataFrame({
    'customers': [loyalty_customers],
    'repeaters': [repeaters],
    'WinbackCustomers': [winback_customers],
    'nonloyalty_txn': [nonloyalty_txn],
    'Nonloyaltysales': [nonloyalty_sales]})

MOM_op['Duration'] = duration
columns = ['Duration'] + [col for col in MOM_op if col != 'Duration']
MOM_output = MOM_op[columns]

In [72]:
## Perform the SQL-like operations using Pandas
# memberdump
# storedump
# skudump
# NLskudump

a = memberdump.merge(storedump, left_on='enrolledstorecode', right_on='storecode', how='left').groupby('region').apply(lambda x: pd.Series({
    'NewEnrollment': x['mobile'].nunique(),
    'WomenEnrolments': (x['gender'] == 'Female').sum(),
    'MenEnrolments': (x['gender'] == 'Male').sum()
}))

b = skudump.merge(storedump, left_on='modifiedstorecode', right_on='storecode', how='left') \
    .groupby('region').apply(lambda x: pd.Series({
    'RedTabSales': x['itemnetamount'].sum(),
    'TotalTransactingCustomers': x['txnmappedmobile'].nunique(),
    'Transacters_1to365D': x[(x['dayssincelastvisit'] >= 1) & (x['dayssincelastvisit'] <= 365)]['txnmappedmobile'].nunique(),
    'Transacters_366to730D': x[(x['dayssincelastvisit'] >= 366) & (x['dayssincelastvisit'] <= 730)]['txnmappedmobile'].nunique(),
    'Winback': x[x['dayssincelastvisit'] > 730]['txnmappedmobile'].nunique(),
    'RedTabTxns': x['modifiedbillno'].nunique()
}))

c = NLskudump.merge(storedump, left_on='modifiedstorecode', right_on='storecode', how='left') \
    .groupby('region').apply(lambda x: pd.Series({
    'NLTxns': x['modifiedbillno'].nunique()
}))

enrolledtransacted = pd.merge(memberdump, skudump, left_on='mobile', right_on='txnmappedmobile', how='inner')[['mobile','enrolledstorecode']]

d = enrolledtransacted.merge(storedump, left_on='enrolledstorecode', right_on='storecode', how='left') \
    .groupby('region').apply(lambda x: pd.Series({
    'NewlyEnrolledTransacted': x['mobile'].nunique()
}))

region_level = pd.concat([a, b, c, d], axis=1)
region_level['TotalTxns'] = region_level['RedTabTxns'] + region_level['NLTxns']

region_level['Duration'] = duration
columns = ['Duration'] + [col for col in region_level if col != 'Duration']
region_level_output = region_level[columns]

In [73]:
#### Importing Tables for Region wise metrics calculations (for previous year)

## Sku report Loyalty
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")

mycursor_SkuDump =SQLdata.cursor()

mycursor_SkuDump =SQLdata.cursor()
mycursor_SkuDump.execute("""SELECT txnmappedmobile,modifiedtxndate,itemnetamount,dayssincelastvisit,modifiedbillno,
modifiedstorecode,frequencycount,itemqty FROM levis.sku_report_loyalty WHERE (modifiedtxndate \
BETWEEN '"""+last_month_start_date_LY+"""' AND '"""+last_month_end_date_LY+"""') AND \
itemqty>0;""")
PYskudump = pd.DataFrame(mycursor_SkuDump)
PYskudump.columns = mycursor_SkuDump.column_names

## Sku report NonLoyalty
mycursor_NLSkuDump =SQLdata.cursor()

mycursor_NLSkuDump =SQLdata.cursor()
mycursor_NLSkuDump.execute("""SELECT modifiedtxndate,modifiedbillno,modifiedstorecode 
FROM levis.sku_report_nonloyalty WHERE (modifiedtxndate \
BETWEEN '"""+last_month_start_date_LY+"""' AND '"""+last_month_end_date_LY+"""') AND itemqty>0;""")
PYNLskudump = pd.DataFrame(mycursor_NLSkuDump)
PYNLskudump.columns = mycursor_NLSkuDump.column_names

## Member report
mycursor_memberreport =SQLdata.cursor()

mycursor_memberreport.execute("""SELECT mobile,gender,enrolledstorecode FROM levis.member_report WHERE modifiedenrolledon \
BETWEEN '"""+last_month_start_date_LY+"""' AND '"""+last_month_end_date_LY+"""';""")

PYmemberdump = pd.DataFrame(mycursor_memberreport)
PYmemberdump.columns = mycursor_memberreport.column_names

In [74]:
## Perform sql like calculations and aggregations with python functions 

a = PYmemberdump.merge(storedump, left_on='enrolledstorecode', right_on='storecode', how='left').groupby('region').apply(lambda x: pd.Series({
    'NewEnrollment': x['mobile'].nunique(),
    'WomenEnrolments': (x['gender'] == 'Female').sum(),
    'MenEnrolments': (x['gender'] == 'Male').sum()
}))

b = PYskudump.merge(storedump, left_on='modifiedstorecode', right_on='storecode', how='left') \
    .groupby('region').apply(lambda x: pd.Series({
    'RedTabSales': x['itemnetamount'].sum(),
    'TotalTransactingCustomers': x['txnmappedmobile'].nunique(),
    'Transacters_1to365D': x[(x['dayssincelastvisit'] >= 1) & (x['dayssincelastvisit'] <= 365)]['txnmappedmobile'].nunique(),
    'Transacters_366to730D': x[(x['dayssincelastvisit'] >= 366) & (x['dayssincelastvisit'] <= 730)]['txnmappedmobile'].nunique(),
    'Winback': x[x['dayssincelastvisit'] > 730]['txnmappedmobile'].nunique(),
    'RedTabTxns': x['modifiedbillno'].nunique()
}))

c = PYNLskudump.merge(storedump, left_on='modifiedstorecode', right_on='storecode', how='left') \
    .groupby('region').apply(lambda x: pd.Series({
    'NLTxns': x['modifiedbillno'].nunique()
}))

PYenrolledtransacted = pd.merge(memberdump, skudump, left_on='mobile', right_on='txnmappedmobile', how='inner')[['mobile','enrolledstorecode']]

d = PYenrolledtransacted.merge(storedump, left_on='enrolledstorecode', right_on='storecode', how='left') \
    .groupby('region').apply(lambda x: pd.Series({
    'NewlyEnrolledTransacted': x['mobile'].nunique()
}))

PY_region_level = pd.concat([a, b, c, d], axis=1)
PY_region_level['TotalTxns'] = PY_region_level['RedTabTxns'] + PY_region_level['NLTxns']

PY_region_level['Duration'] = duration_PY
columns = ['Duration'] + [col for col in PY_region_level if col != 'Duration']
PY_region_level_output = PY_region_level[columns]

In [75]:
## Gender tag
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")
mycursor_Gendertag =SQLdata.cursor()

mycursor_Gendertag.execute("""SELECT txnmappedmobile,itemqty,uniqueitemcode,modifiedtxndate FROM levis.sku_report_loyalty \
WHERE txnmappedmobile in \
(select txnmappedmobile from levis.sku_report_loyalty where (modifiedtxndate between \
'"""+last_month_start_date+"""' AND '"""+last_month_end_date+"""') and itemqty>0) \
and modifiedtxndate<='"""+last_month_end_date+"""' AND itemqty>0;""")

GenderDump = pd.DataFrame(mycursor_Gendertag)
GenderDump.columns = mycursor_Gendertag.column_names

mycursor_Gendertag.execute("""SELECT uniqueitemcode,gender FROM levis.item_master;""")

itemmaster = pd.DataFrame(mycursor_Gendertag)
itemmaster.columns = mycursor_Gendertag.column_names

GenderDumpUC = pd.merge(GenderDump, itemmaster, on='uniqueitemcode', how='left')[['txnmappedmobile','itemqty','uniqueitemcode',
                                                                   'modifiedtxndate','gender']]

## Monthly memberdump
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")
mycursor_monthlymember =SQLdata.cursor()

mycursor_monthlymember.execute("""SELECT mobile,gender FROM levis.member_report where mobile in \
(SELECT DISTINCT txnmappedmobile FROM levis.sku_report_loyalty where(modifiedtxndate between \
'"""+last_month_start_date+"""' AND '"""+last_month_end_date+"""') and itemqty>0) \
group by 1;""")

monthlymember = pd.DataFrame(mycursor_monthlymember)
monthlymember.columns = mycursor_monthlymember.column_names

In [76]:
## Apply the conditions and perform the aggregations
## Python never shows null and blank values count by normal code, check below code for null and blank counts
#GenderDumpUC[GenderDumpUC['gender'].isnull()] #to see null values in gender column
#GenderDumpUC[GenderDumpUC['gender'] == ''] #to see blank values in gender column
GenderDumpUC['gender'].fillna('notag', inplace=True)

women_condition = GenderDumpUC['gender'].str.contains('WOMENS|Women|Total Womens', case=False)
men_condition = (~women_condition) & GenderDumpUC['gender'].str.contains('Mens|Men|Total Mens', case=False)

gender_output = GenderDumpUC.groupby('txnmappedmobile').agg(
    womenqty=('itemqty', lambda x: x[women_condition].sum()),
    menqty=('itemqty', lambda x: x[men_condition].sum()),
    totalqty=('itemqty', 'sum')
).reset_index()

gender_output['womenper'] = gender_output['womenqty'] / gender_output['totalqty'].replace(0, pd.NA)
gender_output['menper'] = gender_output['menqty'] / gender_output['totalqty'].replace(0, pd.NA)

In [77]:
## Updating customer gender
membergender = pd.merge(gender_output, monthlymember, left_on='txnmappedmobile', right_on='mobile', how='left')
gender_output['gender'] = membergender['gender']

## Updating dslv from skudump
skudump_unique_dslv = skudump.groupby('txnmappedmobile')['dayssincelastvisit'].min().reset_index()

dslv_merger = pd.merge(gender_output, skudump_unique_dslv, on='txnmappedmobile', how='left')
gender_output['dayssincelastvisit'] = dslv_merger['dayssincelastvisit']


In [78]:
## Tagging Customer gender to 'p1','p2'
gender_output.loc[gender_output['gender'].str.contains('FEMALE|female', case=False, na=False), 'seg'] = 'p2'
gender_output.loc[(gender_output['gender'].str.contains('MALE|male', case=False, na=False)) & (gender_output['seg'].isnull()), 'seg'] = 'p1'

# Define conditions using NumPy for comparing floating-point numbers (5180 nulls: menpr 1070)
condition_p3 = np.logical_and(gender_output['menper'] > 0.7, gender_output['seg'].isnull())
condition_p4 = np.logical_and(gender_output['womenper'] > 0.7, gender_output['seg'].isnull())
condition_others = gender_output['seg'].isnull()

# Update 'seg' column based on conditions using NumPy's boolean indexing
gender_output.loc[condition_p3, 'seg'] = 'p3'
gender_output.loc[condition_p4 & ~condition_p3, 'seg'] = 'p4'
gender_output.loc[condition_others & ~condition_p3 & ~condition_p4, 'seg'] = 'others'
#gender_output.drop(['seg'],axis=1, inplace=True)

In [79]:
## Define the conditions and corresponding labels for the segmentation
gender_output['dayssincelastvisit'].fillna(0, inplace=True)

conditions = [
    (gender_output['dayssincelastvisit'].between(1, 730, inclusive='both')),
    (gender_output['dayssincelastvisit'] > 730),
    (gender_output['dayssincelastvisit'] < 1)
]
tags = ['ActiveCustomer','LapsedCustomer','NewCustomer']

gender_output['lapsation'] = None

## Update the 'lapsation' column based on conditions
for condition, lapsation in zip(conditions, tags):
    gender_output.loc[condition, 'lapsation'] = lapsation

## Output
itemgender_output = gender_output.groupby(['seg', 'lapsation']).agg({'txnmappedmobile': 'nunique'}).reset_index()

itemgender_output['Duration'] = duration
columns = ['Duration'] + [col for col in itemgender_output if col != 'Duration']
itemgender_finaloutput = itemgender_output[columns]

In [80]:
## Lifecycle split
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")

## Last Month LC table
mycursor_LC_lastmonth =SQLdata.cursor()
mycursor_LC_lastmonth.execute("""SELECT mobile,tag FROM """+last_month_LC+""" GROUP BY 1;""")
LC_lastmonth = pd.DataFrame(mycursor_LC_lastmonth)
LC_lastmonth.columns = mycursor_LC_lastmonth.column_names

## Current Month LC table
mycursor_LC_currentmonth =SQLdata.cursor()
mycursor_LC_currentmonth.execute("""SELECT mobile,tag FROM """+current_month_LC+""" GROUP BY 1;""")
LC_currentmonth = pd.DataFrame(mycursor_LC_currentmonth)
LC_currentmonth.columns = mycursor_LC_currentmonth.column_names

## Counts and transacted current month
last_month_LC_count = LC_lastmonth.groupby('tag').size().reset_index(name='Total Base in Last Month LC')
current_month_LC_count = LC_currentmonth.groupby('tag').size().reset_index(name='Total Base in Current Month LC')

transcated_LC = LC_lastmonth[LC_lastmonth['mobile'].isin(skudump_unique_dslv['txnmappedmobile'])] # using this unique mobile table --> skudump_unique_dslv
customerstransacted_fromlastmonthLC = transcated_LC.groupby('tag').size().reset_index(name='Customers transacted in Current Month from Last Month LC')

lastcurrent = pd.merge(last_month_LC_count, current_month_LC_count, on='tag', how='inner')
LC_output = pd.merge(lastcurrent, customerstransacted_fromlastmonthLC, on='tag', how='left')

LC_output['Duration'] = duration
columns = ['Duration'] + [col for col in LC_output if col != 'Duration']
LC_finaloutput = LC_output[columns]

## Migration
joined_data = pd.merge(LC_lastmonth, LC_currentmonth, on='mobile', suffixes=('_LastMonth', '_RecentMonth'))
migration_output = joined_data.groupby(['tag_LastMonth', 'tag_RecentMonth']).size().reset_index(name='Count')

migration_output['Duration'] = duration
columns = ['Duration'] + [col for col in migration_output if col != 'Duration']
migration_finaloutput = migration_output[columns]

In [81]:
## Discount Metrics
merged_df = pd.merge(LC_lastmonth[['mobile','tag']], skudump[['txnmappedmobile','itemmrp','itemqty','itemnetamount']], how='inner', left_on='mobile', right_on='txnmappedmobile')
merged_df['fullamount'] = merged_df['itemmrp'] * merged_df['itemqty']
merged_df_unique = merged_df.groupby(['mobile', 'tag']).agg(mrp=('fullamount', 'sum'),
                                                            itemamount=('itemnetamount', 'sum')).reset_index()
merged_df_unique['disc_per'] = (merged_df_unique['mrp'] - merged_df_unique['itemamount']) / merged_df_unique['mrp']

# Define a function to categorize 'disc_per' into discount brackets
#merged_df_unique.drop(['disc_bracket'],axis=1, inplace=True)

tolerance = 1e-10

conditions = [
    merged_df_unique['disc_per'] <= 0.01,
    (merged_df_unique['disc_per'] > 0.01) & (merged_df_unique['disc_per'] - tolerance <= 0.25),
    (merged_df_unique['disc_per'] > 0.25) & (merged_df_unique['disc_per'] - tolerance <= 0.40),
    (merged_df_unique['disc_per'] > 0.40) & (merged_df_unique['disc_per'] - tolerance <= 0.50),
    (merged_df_unique['disc_per'] > 0.50) & (merged_df_unique['disc_per'] - tolerance <= 0.60 + tolerance),
    (merged_df_unique['disc_per'] - tolerance) > 0.60
]

# Define the labels for each condition
labels = ['Fresh', '0-25%', '26-40%', '41-50%', '51-60%', '>60%']

# Use NumPy's select function to assign labels based on conditions
merged_df_unique['disc_bracket'] = np.select(conditions, labels, default=np.nan)

# Group by 'tag' and 'disc_bracket', and count distinct 'mobile' values
discount_metrics_output = (
    merged_df_unique.groupby(['tag', 'disc_bracket'])
    .agg(Cust=('mobile', 'nunique'))
    .reset_index()
)

discount_metrics_output['Duration'] = duration
columns = ['Duration'] + [col for col in discount_metrics_output if col != 'Duration']
discount_metrics_finaloutput = discount_metrics_output[columns]

#merged_df_unique.drop(['disc_bracket'],axis=1, inplace=True)

# Snapshot

In [ ]:
## Importing 
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")

## Member Report
mycursor_member_snap =SQLdata.cursor()
mycursor_member_snap.execute("""SELECT mobile,modifiedenrolledon,gender FROM levis.member_report WHERE \
(modifiedenrolledon BETWEEN '"""+last_month2Yback_start_date+"""' AND '"""+last_month2Yback_end_date+"""') OR \
(modifiedenrolledon BETWEEN '"""+last_month_start_date_LY+"""' AND '"""+last_month_end_date_LY+"""') OR \
(modifiedenrolledon BETWEEN '"""+last_2month_start_date+"""' AND '"""+last_2month_end_date+"""') OR \
(modifiedenrolledon BETWEEN '"""+last_month_start_date+"""' AND '"""+last_month_end_date+"""');""")
member_snap = pd.DataFrame(mycursor_member_snap)
member_snap.columns = mycursor_member_snap.column_names
member_snap['modifiedenrolledon'] = pd.to_datetime(member_snap['modifiedenrolledon'])

## Sku Report Loyalty
mycursor_sku_snap =SQLdata.cursor()
mycursor_sku_snap.execute("""SELECT txnmappedmobile,modifiedtxndate,itemnetamount,dayssincelastvisit,modifiedbillno, \
modifiedstorecode,frequencycount,itemqty FROM levis.sku_report_loyalty WHERE 
((modifiedtxndate BETWEEN '"""+last_month2Yback_start_date+"""' AND '"""+last_month2Yback_end_date+"""') OR \
(modifiedtxndate BETWEEN '"""+last_month_start_date_LY+"""' AND '"""+last_month_end_date_LY+"""') OR \
(modifiedtxndate BETWEEN '"""+last_2month_start_date+"""' AND '"""+last_2month_end_date+"""') OR \
(modifiedtxndate BETWEEN '"""+last_month_start_date+"""' AND '"""+last_month_end_date+"""')) and itemqty>0;""")
sku_snap = pd.DataFrame(mycursor_sku_snap)
sku_snap.columns = mycursor_sku_snap.column_names
sku_snap['modifiedtxndate'] = pd.to_datetime(sku_snap['modifiedtxndate'])

## Sku Report NonLoyalty
mycursor_skunonloyalty_snap =SQLdata.cursor()
mycursor_skunonloyalty_snap.execute("""SELECT itemnetamount,modifiedtxndate,modifiedbillno,modifiedstorecode FROM \
levis.sku_report_nonloyalty WHERE 
((modifiedtxndate BETWEEN '"""+last_month2Yback_start_date+"""' AND '"""+last_month2Yback_end_date+"""') OR \
(modifiedtxndate BETWEEN '"""+last_month_start_date_LY+"""' AND '"""+last_month_end_date_LY+"""') OR \
(modifiedtxndate BETWEEN '"""+last_2month_start_date+"""' AND '"""+last_2month_end_date+"""') OR \
(modifiedtxndate BETWEEN '"""+last_month_start_date+"""' AND '"""+last_month_end_date+"""')) AND itemqty>0;""")
skunonloyalty_snap = pd.DataFrame(mycursor_skunonloyalty_snap)
skunonloyalty_snap.columns = mycursor_skunonloyalty_snap.column_names
skunonloyalty_snap['modifiedtxndate'] = pd.to_datetime(skunonloyalty_snap['modifiedtxndate'])

In [ ]:
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")

## 1Yr active a to b
mycursor_1yr_a_b =SQLdata.cursor()
mycursor_1yr_a_b.execute("""SELECT count(distinct txnmappedmobile)Cust from levis.sku_report_loyalty where 
(modifiedtxndate BETWEEN date_sub('"""+last_month2Yback_end_date+"""',interval 1 year) AND '"""+last_month2Yback_end_date+"""') \
and itemqty>0;""")
oneyr_active_a_b = pd.DataFrame(mycursor_1yr_a_b)
oneyr_active_a_b.columns = mycursor_1yr_a_b.column_names

In [ ]:
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")

## 2Yr active a to b
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")
mycursor_2yr_a_b =SQLdata.cursor()
mycursor_2yr_a_b.execute("""SELECT count(distinct txnmappedmobile)Cust from levis.sku_report_loyalty where 
(modifiedtxndate BETWEEN date_sub('"""+last_month2Yback_end_date+"""',interval 2 year) AND '"""+last_month2Yback_end_date+"""') \
and itemqty>0;""")
twoyr_active_a_b = pd.DataFrame(mycursor_2yr_a_b)
twoyr_active_a_b.columns = mycursor_2yr_a_b.column_names

In [ ]:
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")

## 1Yr active c to d
mycursor_1yr_c_d =SQLdata.cursor()
mycursor_1yr_c_d.execute("""SELECT count(distinct txnmappedmobile)Cust from levis.sku_report_loyalty where 
(modifiedtxndate BETWEEN date_sub('"""+last_month_end_date_LY+"""',interval 1 year) AND '"""+last_month_end_date_LY+"""') \
and itemqty>0;""")
oneyr_active_c_d = pd.DataFrame(mycursor_1yr_c_d)
oneyr_active_c_d.columns = mycursor_1yr_c_d.column_names

In [ ]:
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")

## 2Yr active c to d
mycursor_2yr_c_d =SQLdata.cursor()
mycursor_2yr_c_d.execute("""SELECT count(distinct txnmappedmobile)Cust from levis.sku_report_loyalty where 
(modifiedtxndate BETWEEN date_sub('"""+last_month_end_date_LY+"""',interval 2 year) AND '"""+last_month_end_date_LY+"""') \
and itemqty>0;""")
twoyr_active_c_d = pd.DataFrame(mycursor_2yr_c_d)
twoyr_active_c_d.columns = mycursor_2yr_c_d.column_names

In [ ]:
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")

## 1Yr active e to f
mycursor_1yr_e_f =SQLdata.cursor()
mycursor_1yr_e_f.execute("""SELECT count(distinct txnmappedmobile)Cust from levis.sku_report_loyalty where 
(modifiedtxndate BETWEEN date_sub('"""+last_2month_end_date+"""',interval 1 year) AND '"""+last_2month_end_date+"""') \
and itemqty>0;""")
oneyr_active_e_f = pd.DataFrame(mycursor_1yr_e_f)
oneyr_active_e_f.columns = mycursor_1yr_e_f.column_names

In [ ]:
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")

## 2Yr active e to f
mycursor_2yr_e_f =SQLdata.cursor()
mycursor_2yr_e_f.execute("""SELECT count(distinct txnmappedmobile)Cust from levis.sku_report_loyalty where 
(modifiedtxndate BETWEEN date_sub('"""+last_2month_end_date+"""',interval 2 year) AND '"""+last_2month_end_date+"""') \
and itemqty>0;""")
twoyr_active_e_f = pd.DataFrame(mycursor_2yr_e_f)
twoyr_active_e_f.columns = mycursor_2yr_e_f.column_names

In [ ]:
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")

## 1Yr active g to h
mycursor_1yr_g_h =SQLdata.cursor()
mycursor_1yr_g_h.execute("""SELECT count(distinct txnmappedmobile)Cust from levis.sku_report_loyalty where 
(modifiedtxndate BETWEEN date_sub('"""+last_month_end_date+"""',interval 1 year) AND '"""+last_month_end_date+"""') \
and itemqty>0;""")
oneyr_active_g_h = pd.DataFrame(mycursor_1yr_g_h)
oneyr_active_g_h.columns = mycursor_1yr_g_h.column_names

In [ ]:
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")

## 2Yr active g to h
mycursor_2yr_g_h =SQLdata.cursor()
mycursor_2yr_g_h.execute("""SELECT count(distinct txnmappedmobile)Cust from levis.sku_report_loyalty where 
(modifiedtxndate BETWEEN date_sub('"""+last_month_end_date+"""',interval 2 year) AND '"""+last_month_end_date+"""') \
and itemqty>0;""")
twoyr_active_g_h = pd.DataFrame(mycursor_2yr_g_h)
twoyr_active_g_h.columns = mycursor_2yr_g_h.column_names

In [ ]:
# Splitting data  for a-b, c-d, e-f, g-h periods
member_a_b_period = member_snap[(member_snap['modifiedenrolledon'] >= last_month2Yback_start_date) & 
                         (member_snap['modifiedenrolledon'] <= last_month2Yback_end_date)]
member_c_d_period = member_snap[(member_snap['modifiedenrolledon'] >= last_month_start_date_LY) & 
                         (member_snap['modifiedenrolledon'] <= last_month_end_date_LY)]
member_e_f_period = member_snap[(member_snap['modifiedenrolledon'] >= last_2month_start_date) & 
                         (member_snap['modifiedenrolledon'] <= last_2month_end_date)]
member_g_h_period = member_snap[(member_snap['modifiedenrolledon'] >= last_month_start_date) & 
                         (member_snap['modifiedenrolledon'] <= last_month_end_date)]

skuloyal_a_b_period = sku_snap[(sku_snap['modifiedtxndate'] >= last_month2Yback_start_date) & 
                         (sku_snap['modifiedtxndate'] <= last_month2Yback_end_date)]
skuloyal_c_d_period = sku_snap[(sku_snap['modifiedtxndate'] >= last_month_start_date_LY) & 
                         (sku_snap['modifiedtxndate'] <= last_month_end_date_LY)]
skuloyal_e_f_period = sku_snap[(sku_snap['modifiedtxndate'] >= last_2month_start_date) & 
                         (sku_snap['modifiedtxndate'] <= last_2month_end_date)]
skuloyal_g_h_period = sku_snap[(sku_snap['modifiedtxndate'] >= last_month_start_date) & 
                         (sku_snap['modifiedtxndate'] <= last_month_end_date)]

skunonloyal_a_b_period = skunonloyalty_snap[(skunonloyalty_snap['modifiedtxndate'] >= last_month2Yback_start_date) & 
                         (skunonloyalty_snap['modifiedtxndate'] <= last_month2Yback_end_date)]
skunonloyal_c_d_period = skunonloyalty_snap[(skunonloyalty_snap['modifiedtxndate'] >= last_month_start_date_LY) & 
                         (skunonloyalty_snap['modifiedtxndate'] <= last_month_end_date_LY)]
skunonloyal_e_f_period = skunonloyalty_snap[(skunonloyalty_snap['modifiedtxndate'] >= last_2month_start_date) & 
                         (skunonloyalty_snap['modifiedtxndate'] <= last_2month_end_date)]
skunonloyal_g_h_period = skunonloyalty_snap[(skunonloyalty_snap['modifiedtxndate'] >= last_month_start_date) & 
                         (skunonloyalty_snap['modifiedtxndate'] <= last_month_end_date)]

In [ ]:
## Enrollments Split
new_enrollments_a_b = member_a_b_period.shape[0]
new_enrollments_c_d = member_c_d_period.shape[0]
new_enrollments_e_f = member_e_f_period.shape[0]
new_enrollments_g_h = member_g_h_period.shape[0]

female_enrollments_a_b = member_a_b_period[(member_a_b_period['gender'].str.contains('female|Female|FEMALE|Women',case=False, na=False)) &
                                           (member_a_b_period['gender'].notna())]['mobile'].nunique()
female_enrollments_c_d = member_c_d_period[(member_c_d_period['gender'].str.contains('female|Female|FEMALE|Women',case=False, na=False)) &
                                           (member_c_d_period['gender'].notna())]['mobile'].nunique()                                    
female_enrollments_e_f = member_e_f_period[(member_e_f_period['gender'].str.contains('female|Female|FEMALE|Women',case=False, na=False)) &
                                           (member_e_f_period['gender'].notna())]['mobile'].nunique()
female_enrollments_g_h = member_g_h_period[(member_g_h_period['gender'].str.contains('female|Female|FEMALE|Women',case=False, na=False)) &
                                           (member_g_h_period['gender'].notna())]['mobile'].nunique()

male_enrollments_a_b = member_a_b_period[(member_a_b_period['gender'].str.contains('male|Male|MALE|Men',case=False, na=False)) &
                                         (member_a_b_period['gender'].notna())]['mobile'].nunique()
male_enrollments_c_d = member_c_d_period[(member_c_d_period['gender'].str.contains('male|Male|MALE|Men',case=False, na=False)) &
                                         (member_c_d_period['gender'].notna())]['mobile'].nunique()
male_enrollments_e_f = member_e_f_period[(member_e_f_period['gender'].str.contains('male|Male|MALE|Men',case=False, na=False)) &
                                         (member_e_f_period['gender'].notna())]['mobile'].nunique()
male_enrollments_g_h = member_g_h_period[(member_g_h_period['gender'].str.contains('male|Male|MALE|Men',case=False, na=False)) &
                                         (member_g_h_period['gender'].notna())]['mobile'].nunique()

In [ ]:
## Enrollments Combined
new_comb = pd.DataFrame({'Period': [duration_2Y_back, duration_PY, duration_lasttolast_month, duration],
        'New Enrollments': [new_enrollments_a_b, new_enrollments_c_d, new_enrollments_e_f, new_enrollments_g_h]})
male_comb = pd.DataFrame({'Period': [duration_2Y_back, duration_PY, duration_lasttolast_month, duration],
        'Male Enrollments': [male_enrollments_a_b, male_enrollments_c_d, male_enrollments_e_f, male_enrollments_g_h]})
female_comb = pd.DataFrame({'Period': [duration_2Y_back, duration_PY, duration_lasttolast_month, duration],
        'Female Enrollments': [female_enrollments_a_b, female_enrollments_c_d, female_enrollments_e_f, female_enrollments_g_h]})

In [ ]:
## Metrics for period a to b 
red_tab_sales_a_b = skuloyal_a_b_period['itemnetamount'].sum()
total_transacting_customers_a_b = skuloyal_a_b_period['txnmappedmobile'].nunique()
winback_a_b = skuloyal_a_b_period.loc[skuloyal_a_b_period['dayssincelastvisit'] > 730, 'txnmappedmobile'].nunique()
transacters_1_365d_a_b = skuloyal_a_b_period.loc[(skuloyal_a_b_period['dayssincelastvisit'] >= 1) & (skuloyal_a_b_period['dayssincelastvisit'] <= 365), 'txnmappedmobile'].nunique()
transacters_366_730d_a_b = skuloyal_a_b_period.loc[(skuloyal_a_b_period['dayssincelastvisit'] >= 366) & (skuloyal_a_b_period['dayssincelastvisit'] <= 730), 'txnmappedmobile'].nunique()
red_tab_txns_a_b = skuloyal_a_b_period.groupby(['modifiedtxndate', 'modifiedbillno', 'modifiedstorecode']).ngroups
repeat_txn_a_b = skuloyal_a_b_period.loc[skuloyal_a_b_period['frequencycount'] > 1, ['modifiedtxndate', 'modifiedbillno', 'modifiedstorecode']].astype(str).agg(''.join, axis=1).nunique()
abv_a_b = red_tab_sales_a_b / red_tab_txns_a_b

nonloyal_sales_a_b = skunonloyal_a_b_period['itemnetamount'].sum()
nonloyal_txns_a_b = skunonloyal_a_b_period.groupby(['modifiedtxndate', 'modifiedbillno', 'modifiedstorecode']).ngroups

In [ ]:
## Metrics for period c to d 
red_tab_sales_c_d = skuloyal_c_d_period['itemnetamount'].sum()
total_transacting_customers_c_d = skuloyal_c_d_period['txnmappedmobile'].nunique()
winback_c_d = skuloyal_c_d_period.loc[skuloyal_c_d_period['dayssincelastvisit'] > 730, 'txnmappedmobile'].nunique()
transacters_1_365d_c_d = skuloyal_c_d_period.loc[(skuloyal_c_d_period['dayssincelastvisit'] >= 1) & (skuloyal_c_d_period['dayssincelastvisit'] <= 365), 'txnmappedmobile'].nunique()
transacters_366_730d_c_d = skuloyal_c_d_period.loc[(skuloyal_c_d_period['dayssincelastvisit'] >= 366) & (skuloyal_c_d_period['dayssincelastvisit'] <= 730), 'txnmappedmobile'].nunique()
red_tab_txns_c_d = skuloyal_c_d_period.groupby(['modifiedtxndate', 'modifiedbillno', 'modifiedstorecode']).ngroups
repeat_txn_c_d = skuloyal_c_d_period.loc[skuloyal_c_d_period['frequencycount'] > 1, ['modifiedtxndate', 'modifiedbillno', 'modifiedstorecode']].astype(str).agg(''.join, axis=1).nunique()
abv_c_d = red_tab_sales_c_d / red_tab_txns_c_d

nonloyal_sales_c_d = skunonloyal_c_d_period['itemnetamount'].sum()
nonloyal_txns_c_d = skunonloyal_c_d_period.groupby(['modifiedtxndate', 'modifiedbillno', 'modifiedstorecode']).ngroups

In [ ]:
## Metrics for period e to f 
red_tab_sales_e_f = skuloyal_e_f_period['itemnetamount'].sum()
total_transacting_customers_e_f = skuloyal_e_f_period['txnmappedmobile'].nunique()
winback_e_f = skuloyal_e_f_period.loc[skuloyal_e_f_period['dayssincelastvisit'] > 730, 'txnmappedmobile'].nunique()
transacters_1_365d_e_f = skuloyal_e_f_period.loc[(skuloyal_e_f_period['dayssincelastvisit'] >= 1) & (skuloyal_e_f_period['dayssincelastvisit'] <= 365), 'txnmappedmobile'].nunique()
transacters_366_730d_e_f = skuloyal_e_f_period.loc[(skuloyal_e_f_period['dayssincelastvisit'] >= 366) & (skuloyal_e_f_period['dayssincelastvisit'] <= 730), 'txnmappedmobile'].nunique()
red_tab_txns_e_f = skuloyal_e_f_period.groupby(['modifiedtxndate', 'modifiedbillno', 'modifiedstorecode']).ngroups
repeat_txn_e_f = skuloyal_e_f_period.loc[skuloyal_e_f_period['frequencycount'] > 1, ['modifiedtxndate', 'modifiedbillno', 'modifiedstorecode']].astype(str).agg(''.join, axis=1).nunique()
abv_e_f = red_tab_sales_e_f / red_tab_txns_e_f

nonloyal_sales_e_f = skunonloyal_e_f_period['itemnetamount'].sum()
nonloyal_txns_e_f = skunonloyal_e_f_period.groupby(['modifiedtxndate', 'modifiedbillno', 'modifiedstorecode']).ngroups

In [ ]:
## Metrics for period g to h
red_tab_sales_g_h = skuloyal_g_h_period['itemnetamount'].sum()
total_transacting_customers_g_h = skuloyal_g_h_period['txnmappedmobile'].nunique()
winback_g_h = skuloyal_g_h_period.loc[skuloyal_g_h_period['dayssincelastvisit'] > 730, 'txnmappedmobile'].nunique()
transacters_1_365d_g_h = skuloyal_g_h_period.loc[(skuloyal_g_h_period['dayssincelastvisit'] >= 1) & (skuloyal_g_h_period['dayssincelastvisit'] <= 365), 'txnmappedmobile'].nunique()
transacters_366_730d_g_h = skuloyal_g_h_period.loc[(skuloyal_g_h_period['dayssincelastvisit'] >= 366) & (skuloyal_g_h_period['dayssincelastvisit'] <= 730), 'txnmappedmobile'].nunique()
red_tab_txns_g_h = skuloyal_g_h_period.groupby(['modifiedtxndate', 'modifiedbillno', 'modifiedstorecode']).ngroups
repeat_txn_g_h = skuloyal_g_h_period.loc[skuloyal_g_h_period['frequencycount'] > 1, ['modifiedtxndate', 'modifiedbillno', 'modifiedstorecode']].astype(str).agg(''.join, axis=1).nunique()
abv_g_h = red_tab_sales_g_h / red_tab_txns_g_h

nonloyal_sales_g_h = skunonloyal_g_h_period['itemnetamount'].sum()
nonloyal_txns_g_h = skunonloyal_g_h_period.groupby(['modifiedtxndate', 'modifiedbillno', 'modifiedstorecode']).ngroups

In [ ]:
## Enrolled & Transacted
enrolled_transacted_a_b = pd.merge(member_a_b_period, skuloyal_a_b_period, left_on='mobile', right_on='txnmappedmobile', how='inner')
enrolled_transacted_c_d = pd.merge(member_c_d_period, skuloyal_c_d_period, left_on='mobile', right_on='txnmappedmobile', how='inner')
enrolled_transacted_e_f = pd.merge(member_e_f_period, skuloyal_e_f_period, left_on='mobile', right_on='txnmappedmobile', how='inner')
enrolled_transacted_g_h = pd.merge(member_g_h_period, skuloyal_g_h_period, left_on='mobile', right_on='txnmappedmobile', how='inner')

enrolled_transacted_unique_a_b = enrolled_transacted_a_b['mobile'].nunique()
enrolled_transacted_unique_c_d = enrolled_transacted_c_d['mobile'].nunique()
enrolled_transacted_unique_e_f = enrolled_transacted_e_f['mobile'].nunique()
enrolled_transacted_unique_g_h = enrolled_transacted_g_h['mobile'].nunique()

In [ ]:
## Lapsed Base 
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")

## Member Report
count_till_date =SQLdata.cursor()
count_till_date.execute("""SELECT txnmappedmobile,modifiedtxndate FROM levis.sku_report_loyalty WHERE \
modifiedtxndate<= '"""+last_month_end_date+"""' and itemqty>0 group by 1,2;""")
totalbase_snap = pd.DataFrame(count_till_date)
totalbase_snap.columns = count_till_date.column_names
totalbase_snap['modifiedtxndate'] = pd.to_datetime(totalbase_snap['modifiedtxndate'])

In [ ]:
## Sorting 'totalBase till date'
totalbase_snap.sort_values(by='modifiedtxndate', inplace=True)

# Set 'transaction_date' as the index for faster lookup
totalbase_snap.set_index('modifiedtxndate', inplace=True)

# Base till date
unique_customers_b = totalbase_snap.loc[:last_month2Yback_end_date, 'txnmappedmobile'].nunique()
unique_customers_d = totalbase_snap.loc[:last_month_end_date_LY, 'txnmappedmobile'].nunique()
unique_customers_f = totalbase_snap.loc[:last_2month_end_date, 'txnmappedmobile'].nunique()
unique_customers_h = totalbase_snap.loc[:last_month_end_date, 'txnmappedmobile'].nunique()

In [ ]:
# Summarizing period wise KPIs
loyaltyKPIs_a_b = pd.DataFrame({
    'New Enrollments' : [new_enrollments_a_b],
    'Men Enrollments' : [male_enrollments_a_b],
    'Women Enrollments' : [female_enrollments_a_b],
    'Total Sales' : [nonloyal_sales_a_b + red_tab_sales_a_b],
    'Red Tab Sales': [red_tab_sales_a_b],
    '% of Red Tab Sales': [(red_tab_sales_a_b / (nonloyal_sales_a_b + red_tab_sales_a_b)) * 100],
    'Total Transacting Customers': [total_transacting_customers_a_b],
    'Lapsed Customer till date' : (unique_customers_b - twoyr_active_a_b['Cust'].iloc[0]),
    'Newly Enrolled and Transacted': [enrolled_transacted_unique_a_b],
    'Transacters_1_365D': [transacters_1_365d_a_b],
    'Transacters_366_730D': [transacters_366_730d_a_b],
    'Winback': [winback_a_b],
    'Red Tab Txns': [red_tab_txns_a_b],
    'Total Txns': [red_tab_txns_a_b + nonloyal_txns_a_b],
    'Repeat Txn': [repeat_txn_a_b],
    '1 Year Active Customers': [oneyr_active_a_b['Cust'].iloc[0]],
    '2 Year Active Customers': [twoyr_active_a_b['Cust'].iloc[0]],
    'abv': [abv_a_b],
    '% Repeat Transaction': [(repeat_txn_a_b / red_tab_txns_a_b) * 100]
})
loyaltyKPIs_c_d = pd.DataFrame({
    'New Enrollments' : [new_enrollments_c_d],
    'Men Enrollments' : [male_enrollments_c_d],
    'Women Enrollments' : [female_enrollments_c_d],
    'Total Sales' : [nonloyal_sales_c_d + red_tab_sales_c_d],
    'Red Tab Sales': [red_tab_sales_c_d],
    '% of Red Tab Sales': [(red_tab_sales_c_d / (nonloyal_sales_c_d + red_tab_sales_c_d)) * 100],
    'Total Transacting Customers': [total_transacting_customers_c_d],
    'Lapsed Customer till date' : (unique_customers_d - twoyr_active_c_d['Cust'].iloc[0]),    
    'Newly Enrolled and Transacted': [enrolled_transacted_unique_c_d],
    'Transacters_1_365D': [transacters_1_365d_c_d],
    'Transacters_366_730D': [transacters_366_730d_c_d],
    'Winback': [winback_c_d],
    'Red Tab Txns': [red_tab_txns_c_d],
    'Total Txns': [red_tab_txns_c_d + nonloyal_txns_c_d],
    'Repeat Txn': [repeat_txn_c_d],
    '1 Year Active Customers': [oneyr_active_c_d['Cust'].iloc[0]],
    '2 Year Active Customers': [twoyr_active_c_d['Cust'].iloc[0]],
    'abv': [abv_c_d],
    '% Repeat Transaction': [(repeat_txn_c_d / red_tab_txns_c_d) * 100]
})
loyaltyKPIs_e_f = pd.DataFrame({
    'New Enrollments' : [new_enrollments_e_f],
    'Men Enrollments' : [male_enrollments_e_f],
    'Women Enrollments' : [female_enrollments_e_f],
    'Total Sales' : [nonloyal_sales_e_f + red_tab_sales_e_f],
    'Red Tab Sales': [red_tab_sales_e_f],
    '% of Red Tab Sales': [(red_tab_sales_e_f / (nonloyal_sales_e_f + red_tab_sales_e_f)) * 100],
    'Total Transacting Customers': [total_transacting_customers_e_f],
    'Lapsed Customer till date' : (unique_customers_f - twoyr_active_e_f['Cust'].iloc[0]),    
    'Newly Enrolled and Transacted': [enrolled_transacted_unique_e_f],
    'Transacters_1_365D': [transacters_1_365d_e_f],
    'Transacters_366_730D': [transacters_366_730d_e_f],
    'Winback': [winback_e_f],
    'Red Tab Txns': [red_tab_txns_e_f],
    'Total Txns': [red_tab_txns_e_f + nonloyal_txns_e_f],
    'Repeat Txn': [repeat_txn_e_f],
    '1 Year Active Customers': [oneyr_active_e_f['Cust'].iloc[0]],
    '2 Year Active Customers': [twoyr_active_e_f['Cust'].iloc[0]],
    'abv': [abv_e_f],
    '% Repeat Transaction': [(repeat_txn_e_f / red_tab_txns_e_f) * 100]
})
loyaltyKPIs_g_h = pd.DataFrame({
    'New Enrollments' : [new_enrollments_g_h],
    '% of Red Tab Sales': [(red_tab_sales_g_h / (nonloyal_sales_g_h + red_tab_sales_g_h)) * 100],
    'Men Enrollments' : [male_enrollments_g_h],
    'Women Enrollments' : [female_enrollments_g_h],    
    'Total Sales' : [nonloyal_sales_g_h + red_tab_sales_g_h],
    'Red Tab Sales': [red_tab_sales_g_h],
    'Total Transacting Customers': [total_transacting_customers_g_h],
    'Lapsed Customer till date' : (unique_customers_h - twoyr_active_g_h['Cust'].iloc[0]),    
    'Newly Enrolled and Transacted': [enrolled_transacted_unique_g_h],
    'Transacters_1_365D': [transacters_1_365d_g_h],
    'Transacters_366_730D': [transacters_366_730d_g_h],
    'Winback': [winback_g_h],
    'Red Tab Txns': [red_tab_txns_g_h],
    'Total Txns': [red_tab_txns_g_h + nonloyal_txns_g_h],
    'Repeat Txn': [repeat_txn_g_h],
    '1 Year Active Customers': [oneyr_active_g_h['Cust'].iloc[0]],
    '2 Year Active Customers': [twoyr_active_g_h['Cust'].iloc[0]],
    'abv': [abv_g_h],
    '% Repeat Transaction': [(repeat_txn_g_h / red_tab_txns_g_h) * 100]
})

In [ ]:
## Adding 'Duration' to summarized KPIs
loyaltyKPIs_a_b['Duration'] = duration_2Y_back
columns = ['Duration'] + [col for col in loyaltyKPIs_a_b if col != 'Duration']
loyaltyKPIs_a_b_output = loyaltyKPIs_a_b[columns]

loyaltyKPIs_c_d['Duration'] = duration_PY
columns = ['Duration'] + [col for col in loyaltyKPIs_c_d if col != 'Duration']
loyaltyKPIs_c_d_output = loyaltyKPIs_c_d[columns]

loyaltyKPIs_e_f['Duration'] = duration_lasttolast_month
columns = ['Duration'] + [col for col in loyaltyKPIs_e_f if col != 'Duration']
loyaltyKPIs_e_f_output = loyaltyKPIs_e_f[columns]

loyaltyKPIs_g_h['Duration'] = duration
columns = ['Duration'] + [col for col in loyaltyKPIs_g_h if col != 'Duration']
loyaltyKPIs_g_h_output = loyaltyKPIs_g_h[columns]

In [ ]:
snapshot_op = pd.concat([loyaltyKPIs_a_b_output, loyaltyKPIs_c_d_output, loyaltyKPIs_e_f_output, loyaltyKPIs_g_h_output], ignore_index=True)
#combined_df = snapshot_op.reindex(columns=loyaltyKPIs_a_b_output.columns)
snapshot_output = snapshot_op.transpose()

In [83]:
## Putting pandas file to tabs in excel file
with pd.ExcelWriter('D:/OneDrive - EasyRewardz Software Services Private Limited/Data from D/Levis India MBR Auto '+str(last_month_start_date)+' '+str(last_month_end_date)+'.xlsx') as writer:
    snapshot_output.to_excel(writer, sheet_name='Snapshot', index=False)
    enrollments_output.to_excel(writer, sheet_name='Enrollments', index=False)
    MOM_output.to_excel(writer, sheet_name='MOM', index=False)
    region_level_output.to_excel(writer, sheet_name='Region CY', index=True)
    PY_region_level_output.to_excel(writer, sheet_name='Region PY', index=True)
    winback_output.to_excel(writer, sheet_name='Winback', index=False)
    itemgender_finaloutput.to_excel(writer, sheet_name='Gender', index=False)
    LC_finaloutput.to_excel(writer, sheet_name='LC', index=False)
    migration_finaloutput.to_excel(writer, sheet_name='migration', index=False)
    discount_metrics_finaloutput.to_excel(writer, sheet_name='Discount', index=False)
